In [25]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm, notebook

In [12]:
consumer_key= ''
consumer_secret= ''
access_token= ''
access_token_secret= ''

In [13]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [14]:
# Define the search term and the date_since date as variables
search_words = "#Pfizervaccine"
date_since = "2021-03-01"

In [21]:
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(500)
#.items(5) will return 5 of the most recent tweets.
tweets

In [22]:

tweets_copy = []
for tweet in tqdm(tweets):
     tweets_copy.append(tweet)

500it [00:41, 12.08it/s]


In [23]:
print(f"new tweets retrieved: {len(tweets_copy)}")


new tweets retrieved: 500


In [27]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [07:17<00:00,  1.14it/s]


In [28]:
tweets_df.head()


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Iain Philpott,London,Fashion & Beauty Photographer/Director/DOP,2010-11-25 14:50:14,312,241,2311,False,2021-03-12 07:59:58,No mention on @BBCBreakfast @BBCNews that now ...,[AstraZeneca],Twitter for iPhone,False
0,Africa Insights,Africa,"Insights, analysis, thought-leadership on issu...",2012-05-22 09:42:46,4254,1787,1666,False,2021-03-12 07:52:35,"RT @bjornhamland: Norway gives away 677,000 do...",None,Twitter for Android,False
0,Stu,"Staffordshire, England",Covid-19 Negative (currently),2019-06-14 09:20:34,1558,2419,11064,False,2021-03-12 07:45:36,“We don’t have any evidence that the vaccine i...,None,Twitter for iPhone,False
0,Natalie Trevaskis,"Melbourne, Victoria","Pharmacist, Scientist, Mother, Runner",2017-11-01 10:41:47,542,431,3753,False,2021-03-12 06:56:07,🎉My husband got his Pfizer vaccine today (and ...,None,Twitter for iPhone,False
0,Easton Marks,NYC,It is of the Lord’s mercies...,2013-07-18 23:44:43,429,366,2407,False,2021-03-12 06:48:05,One in. One to go. Get yours #PfizerVaccine ht...,[PfizerVaccine],Twitter for iPhone,False


In [37]:

# read the old CSV file of collected tweets
tweets_old_df = pd.read_csv("tweets.csv")


In [38]:
#Append the new file to the old file
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)


In [40]:
#Drop duplicates if any
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)


In [41]:
#export data to CSV
tweets_all_df.to_csv("tweets.csv", index=False)